<a href="https://colab.research.google.com/github/MrsIgnis/MOCI/blob/main/MOCI_task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Создание многослойной нейронной сети вручную (без использования готовых библиотек и без обратного распространения ошибки -> веса вычисляются рандомно)**

In [1]:
import numpy as np

In [32]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

In [33]:
def forward_pass(
    X: np.ndarray,
    W1: np.ndarray, b1: np.ndarray,
    W2: np.ndarray, b2: np.ndarray,
    W3: np.ndarray, b3: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:

    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)

    Z3 = np.dot(A2, W3) + b3
    A3 = sigmoid(Z3)

    return A1, A2, A3

In [35]:
def initialize_weights(
    input_neurons: int, hidden_neurons_1: int,
    hidden_neurons_2: int, output_neurons: int
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    np.random.seed(42)
    W1 = np.random.randn(input_neurons, hidden_neurons_1) * 0.1
    b1 = np.zeros((1, hidden_neurons_1))

    W2 = np.random.randn(hidden_neurons_1, hidden_neurons_2) * 0.1
    b2 = np.zeros((1, hidden_neurons_2))

    W3 = np.random.randn(hidden_neurons_2, output_neurons) * 0.1
    b3 = np.zeros((1, output_neurons))

    return W1, b1, W2, b2, W3, b3

In [38]:
def predict(
    X: np.ndarray,
    W1: np.ndarray, b1: np.ndarray,
    W2: np.ndarray, b2: np.ndarray,
    W3: np.ndarray, b3: np.ndarray
) -> np.ndarray:

    _, _, A3 = forward_pass(X, W1, b1, W2, b2, W3, b3)
    return A3

In [58]:
dataset = np.array([
    [22, 2, 1, 1, 38000],
    [27, 3, 2, 1, 42000],
    [34, 5, 4, 2, 47000],
    [40, 8, 5, 3, 53000],
    [45, 10, 6, 4, 60000],
    [50, 12, 6, 5, 62000],
    [26, 3, 3, 2, 44000],
    [36, 7, 4, 3, 49000],
    [42, 9, 5, 4, 55000],
    [48, 11, 6, 4, 58000]
])

In [59]:
X = dataset[:, :-1]
y = dataset[:, -1].reshape(-1, 1) / 100000

In [60]:
input_neurons = X.shape[1]
hidden_neurons_1 = 10
hidden_neurons_2 = 6
output_neurons = 1

In [61]:
W1, b1, W2, b2, W3, b3 = initialize_weights(input_neurons, hidden_neurons_1, hidden_neurons_2, output_neurons)

In [62]:
predictions = predict(X, W1, b1, W2, b2, W3, b3)
predictions = predictions * 100000

In [63]:
print(f"{'Возраст':<10} {'Опыт':<10} {'Образование':<15} {'Стаж':<10} {'Доход в датасете':<20} {'Предсказанный доход':<20}")
for i in range(len(X)):
    print(f"{X[i][0]:<10} {X[i][1]:<10} {X[i][2]:<15} {X[i][3]:<10} {y[i][0]*100000:<20} {predictions[i][0]:<20.2f}")

Возраст    Опыт       Образование     Стаж       Доход в датасете     Предсказанный доход 
22         2          1               1          38000.0              46491.85            
27         3          2               1          42000.0              46498.57            
34         5          4               2          47000.0              46513.98            
40         8          5               3          53000.0              46526.87            
45         10         6               4          60000.0              46533.71            
50         12         6               5          62000.0              46539.25            
26         3          3               2          44000.0              46504.06            
36         7          4               3          49000.0              46525.35            
42         9          5               4          55000.00000000001    46533.64            
48         11         6               4          57999.99999999999    46534.30            